##Using Average Annual Growth to estimate missing missing values

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd 
import numpy as np
import scipy 

pd.set_option('display.max_columns', None)

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt 
import seaborn as sns 

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Artificial Intelligence Thinking/Projects/US.csv")

In [ ]:
data

,Unnamed: 0,country,year,iso_code,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,biofuel_electricity,biofuel_share_elec,biofuel_share_energy,carbon_intensity_elec,coal_cons_change_pct,coal_cons_change_twh,coal_cons_per_capita,coal_consumption,coal_elec_per_capita,coal_electricity,coal_prod_change_pct,coal_prod_change_twh,coal_prod_per_capita,coal_production,coal_share_elec,coal_share_energy,electricity_demand,electricity_generation,energy_cons_change_pct,energy_cons_change_twh,energy_per_capita,energy_per_gdp,fossil_cons_change_pct,fossil_cons_change_twh,fossil_elec_per_capita,fossil_electricity,fossil_energy_per_capita,fossil_fuel_consumption,fossil_share_elec,fossil_share_energy,gas_cons_change_pct,gas_cons_change_twh,gas_consumption,gas_elec_per_capita,gas_electricity,gas_energy_per_capita,gas_prod_change_pct,gas_prod_change_twh,gas_prod_per_capita,gas_production,gas_share_elec,gas_share_energy,greenhouse_gas_emissions,hydro_cons_change_pct,hydro_cons_change_twh,hydro_consumption,hydro_elec_per_capita,hydro_electricity,hydro_energy_per_capita,hydro_share_elec,hydro_share_energy,low_carbon_cons_change_pct,low_carbon_cons_change_twh,low_carbon_consumption,low_carbon_elec_per_capita,low_carbon_electricity,low_carbon_energy_per_capita,low_carbon_share_elec,low_carbon_share_energy,net_elec_imports,net_elec_imports_share_demand,nuclear_cons_change_pct,nuclear_cons_change_twh,nuclear_consumption,nuclear_elec_per_capita,nuclear_electricity,nuclear_energy_per_capita,nuclear_share_elec,nuclear_share_energy,oil_cons_change_pct,oil_cons_change_twh,oil_consumption,oil_elec_per_capita,oil_electricity,oil_energy_per_capita,oil_prod_change_pct,oil_prod_change_twh,oil_prod_per_capita,oil_production,oil_share_elec,oil_share_energy,other_renewable_consumption,other_renewable_electricity,other_renewable_exc_biofuel_electricity,other_renewables_cons_change_pct,other_renewables_cons_change_twh,other_renewables_elec_per_capita,other_renewables_elec_per_capita_exc_biofuel,other_renewables_energy_per_capita,other_renewables_share_elec,other_renewables_share_elec_exc_biofuel,other_renewables_share_energy,per_capita_electricity,primary_energy_consumption,renewables_cons_change_pct,renewables_cons_change_twh,renewables_consumption,renewables_elec_per_capita,renewables_electricity,renewables_energy_per_capita,renewables_share_elec,renewables_share_energy,solar_cons_change_pct,solar_cons_change_twh,solar_consumption,solar_elec_per_capita,solar_electricity,solar_energy_per_capita,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
0,21009,United States,1900,USA,78763706.0,6.139981e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23226.369,1829.395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,937.493,73.840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1252.425,98.646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21010,United States,1901,USA,80336086.0,6.831060e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.757,160.198,24765.869,1989.593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.817,7.988,1018.572,81.828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.078,8.955,1339.383,107.601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21011,United States,1902,USA,81949920.0,6.901269e+11,NaN,NaN,NaN

In [ ]:
data.shape

(122, 129)

In [ ]:
for i in data.isnull().sum():
  print (i,' ',end='')

0  0  0  0  0  3  91  91  90  90  100  100  100  90  100  66  66  65  65  85  85  1  1  0  0  85  65  100  85  66  66  65  68  66  66  85  85  65  65  85  65  66  66  65  85  85  65  1  1  0  0  85  65  100  66  66  65  65  65  65  85  65  66  66  65  65  65  65  85  65  100  100  66  66  65  65  65  65  85  65  66  66  65  85  85  65  1  1  0  0  85  65  65  65  100  66  66  65  100  65  85  100  65  85  65  66  66  65  65  65  65  85  65  84  66  65  65  65  65  85  65  84  66  65  65  65  65  85  65  

In [ ]:
data1 = data.drop(columns = ['year','population','Unnamed: 0','country','iso_code'])

**Annual Percent growth rate**


$$
    average~annual~percentage~growth~rate = 
    ((\frac{f}s)^\frac{1}y -1)*100 
$$
***s*** = Starting Year <br>
***f*** = Final Year <br>
***y*** = f - s <br>

$$
    p = \frac{annual~percentage~growth~rate}{100}\
$$

$$
   Increase:~~~~~~~~~~~~ x = (1 + p)x_0
$$

$$
  x_0 = \frac{x}{1+p}\
$$

$$
   Decrease:~~~~~~~~~~~~ x = (1 - p)x_0
$$

Where **x** is the the current value and **x<sub>0</sub>** is the previous value to be determined. In the case of our problem we want to find the previous value and will use the increase function to determine **x<sub>0</sub>**

In [ ]:
value_coulumns = data1.columns.tolist()

In [ ]:
gdp = value_coulumns[0:1] #has gone up. No NaN values
biofuel = value_coulumns[1:9] # has gone up
carbon = value_coulumns[9:10] #has gone down
coal = value_coulumns[10:22] # has gone down 
electricity = value_coulumns[22:24] # has gone up
energy = value_coulumns[24:28] #has gone up
fossil = value_coulumns[28:36] #has gone up
gas = value_coulumns[36:48] #has gone up 
greenhouse = value_coulumns[48:49] #has gone up 
hydro = value_coulumns[49:57] #has gone up
low_carbon = value_coulumns[57:65] #has gone up
net_elec_import = value_coulumns[65:67] #has gone up 
nuclear = value_coulumns[67:75] #has gone up since 1951. 0 before then
oil = value_coulumns[75:87] #has gone up
other_renewable = value_coulumns[87:98] #has gone up
electricity2 = value_coulumns[98:99] # has gone up
energy2 = value_coulumns[99:100] #has gone up
renewable = value_coulumns[100:108] # has gone up
solar = value_coulumns[108:116] # has gone up
Wind = value_coulumns[116:123] #has gone up

In [ ]:
up = greenhouse + electricity + electricity2 + net_elec_import + energy + energy2 + biofuel + nuclear + oil + fossil + gas + solar + Wind + hydro + low_carbon + renewable + other_renewable
down = carbon + coal

In [ ]:
df = pd.DataFrame()

In [ ]:
test1 = []
for i in range(len(up)):
  testlist = data1[up[i]].values.tolist()
  test1.append(testlist)
t2 =test1.copy()

In [ ]:
import math
for sub in range(len(test1)):
  for l in range(len(test1[sub])):
    if math.isnan(test1[sub][l]): 
      test1[sub][l] = 'nan'

In [ ]:
print (test1)

[['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 2025.02, 1995.62, 2024.87, 2058.14, 2089.84, 2140.68, 2112.12, 2174.65, 2129.32, 1955.08, 2062.37, 1979.4, 1900.9, 1908.54, 1913.98, 1828.12, 1755.22, 1688.83, 1731.2, 1633.22, 1493.43, 1573.98], ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'na

In [ ]:
# List of all years
for _list in range(len(test1)):
  
  years1 = [i for i in range(1900,2022)]
  valuesavailable= []
  yearsavailable= []
  #final start
  fs =[]
  #year final start
  yfs =[]
  #difference between year
  dby =[]
  yeargaps = []
  yearnogaps = []
  valuesgaps =[]
  valuesnogaps =[]
  #Create list of values and years that are not nan
  for i in range(len(test1[_list])):

    if test1[_list][i] != 'nan':
      valuesavailable.append(test1[_list][i])
      yearsavailable.append(years1[i])

  #check for gaps
  #years gap place holder and years no gap place holder
  ygph = []
  #value gap and no gap place holder
  vgph = []
  ygp =[]
  yngph =[]
  #year gap range
  ygr = []
  a = []
  # Find all available years and their values
  if len(yearsavailable) > 0:
    for year in range(len(yearsavailable)):
      
      try:
          ygph.append(yearsavailable[year])
          vgph.append(valuesavailable[year])
      except: 
        pass
    try:
  #Finding year gaps
      for i in range(len(ygph)): 
        ygpha =[]
        j = [year for year in range(ygph[i], ygph[i+1]+1)]
        ygr.append(j)
    except:
      pass
    for _range in ygr:
      for year in range(len(yearsavailable)):
        if yearsavailable[year] in _range:
          if _range not in a:
            a.append(_range)
   
  #Finding the Start and Final values of each gap 
  for ryear in a: 
     yfs.append([ryear[-1], ryear[0]])
  yfs = yfs[::-1]
  if len(yfs) > 0: 
    for vfs in range(len(yfs)):
      vf = yearsavailable.index(yfs[vfs][0])
      vs = yearsavailable.index(yfs[vfs][1])
      fs.append([valuesavailable[vf],valuesavailable[vs]])
      dif = yearsavailable[vf] - yearsavailable[vs]
      dby.append(dif)

#increase formula to determine previous values and plug in 'nan' values before last gap value
  aagr_l =[]
  p_l =[]
  try: 
    for i in range(len(fs)):
      aagr = ((fs[i][0]/fs[i][1])**(1/dby[i])-1)*100
      aagr_l.append(aagr)
      p = aagr/100
      p_l.append(p)
  except:
    pass

  try:
    x = fs[0][0]
    nulls = []
    p = []
    for j in range(len(yfs)):
      r1 = (yfs[j][1])
      r2 = (yfs[j][0])
      wrange = [ r for r in range(r1,r2)]
      
      for i in reversed(range(len(test1[_list]))): 
        if years1[i] in wrange:
          p1 = j
          p.append(p1)
    p = p[::-1]
    while len(p) != len(test1[_list]):
      p.append(0)
  
#increase formula
    for i in reversed(range(len(p))): 
        x_0 = (x/(1+p_l[p[i]]))
        if test1[_list][i] == 'nan':
          x = x_0
        else:
          x = test1[_list][i]
        nulls.append(x)
  
    
    nulls = nulls[::-1]
    nulls = nulls[0:-1]
    n = len(nulls)
    while len(nulls) != len(test1[_list]):
      nulls.append('nan')
  except: 
    nulls = test1[_list]
  nulls1 =[]
  if 'nan' in nulls:
    y = years1[len(nulls)-2] - years1[0]

    x =nulls[0]

    try:
      aagr = ((nulls[-2]/nulls[0])**(1/y)-1)*100
      p = aagr/100
      for i in range(len(test1[_list])):

        x_0 = (x/(1-p))
        if nulls[i] == 'nan':
            x = x_0
        else:
          x = nulls[i]
        nulls1.append(x)
    except:
      pass
      
  for v in range(len(nulls1)):
    if v > t2[_list].index(valuesavailable[-1]):
      nulls1[v] = 'nan'
      
  if len(nulls1) == 0: 
    nulls1 = t2[_list]
    for i in range(len(nulls1)):
      if nulls1[i] == 'nan':
        nulls1[i] = 0
  
  for i in range(len(nulls1)):
    if nulls1[i] == 'nan':
       nulls1[i] = None

  df[up[_list]] = nulls1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:149: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [ ]:
test1 = []
for i in range(len(down)):
  testlist = data1[down[i]].values.tolist()
  test1.append(testlist)
t2 =test1.copy()

In [ ]:
import math
for sub in range(len(test1)):
  for l in range(len(test1[sub])):
    if math.isnan(test1[sub][l]): 
      test1[sub][l] = 'nan'

In [ ]:
# List of all years
for _list in range(len(test1)):
  
  years1 = [i for i in range(1900,2022)]
  valuesavailable= []
  yearsavailable= []
  #final start
  fs =[]
  #year final start
  yfs =[]
  #difference between year
  dby =[]
  yeargaps = []
  yearnogaps = []
  valuesgaps =[]
  valuesnogaps =[]
  #Create list of values and years that are not nan
  for i in range(len(test1[_list])):

    if test1[_list][i] != 'nan':
      valuesavailable.append(test1[_list][i])
      yearsavailable.append(years1[i])

  #check for gaps
  #years gap place holder and years no gap place holder
  ygph = []
  #value gap and no gap place holder
  vgph = []
  ygp =[]
  yngph =[]
  #year gap range
  ygr = []
  a = []
  # Find all available years and their values
  if len(yearsavailable) > 0:
    for year in range(len(yearsavailable)):
      
      try:
          ygph.append(yearsavailable[year])
          vgph.append(valuesavailable[year])
      except: 
        pass
    try:
  #Finding year gaps
      for i in range(len(ygph)): 
        ygpha =[]
        j = [year for year in range(ygph[i], ygph[i+1]+1)]
        ygr.append(j)
    except:
      pass
    for _range in ygr:
      for year in range(len(yearsavailable)):
        if yearsavailable[year] in _range:
          if _range not in a:
            a.append(_range)
   
  #Finding the Start and Final values of each gap 
  for ryear in a: 
     yfs.append([ryear[-1], ryear[0]])
  yfs = yfs[::-1]
  if len(yfs) > 0: 
    for vfs in range(len(yfs)):
      vf = yearsavailable.index(yfs[vfs][0])
      vs = yearsavailable.index(yfs[vfs][1])
      fs.append([valuesavailable[vf],valuesavailable[vs]])
      dif = yearsavailable[vf] - yearsavailable[vs]
      dby.append(dif)

#increase formula to determine previous values and plug in 'nan' values before last gap value
  aagr_l =[]
  p_l =[]
  try: 
    for i in range(len(fs)):
      aagr = ((fs[i][0]/fs[i][1])**(1/dby[i])-1)*100
      aagr_l.append(aagr)
      p = aagr/100
      p_l.append(p)
  except:
    pass

  try:
    x = fs[0][0]
    nulls = []
    p = []
    for j in range(len(yfs)):
      r1 = (yfs[j][1])
      r2 = (yfs[j][0])
      wrange = [ r for r in range(r1,r2)]
      
      for i in reversed(range(len(test1[_list]))): 
        if years1[i] in wrange:
          p1 = j
          p.append(p1)
    p = p[::-1]
    while len(p) != len(test1[_list]):
      p.append(0)
  
#decrease formula
    for i in reversed(range(len(p))): 
        x_0 = (x/(1+p_l[p[i]]))
        if test1[_list][i] == 'nan':
          x = x_0
        else:
          x = test1[_list][i]
        nulls.append(x)
  
    
    nulls = nulls[::-1]
    nulls = nulls[0:-1]
    n = len(nulls)
    while len(nulls) != len(test1[_list]):
      nulls.append('nan')
  except: 
    nulls = test1[_list]
  nulls1 =[]
  if 'nan' in nulls:
    y = years1[len(nulls)-2] - years1[0]

    x =nulls[0]

    try:
      aagr = ((nulls[-2]/nulls[0])**(1/y)-1)*100
      p = aagr/100
      for i in range(len(test1[_list])):

        x_0 = (x/(1+p))
        if nulls[i] == 'nan':
            x = x_0
        else:
          x = nulls[i]
        nulls1.append(x)
    except:
      pass
      
  for v in range(len(nulls1)):
    if v > t2[_list].index(valuesavailable[-1]):
      nulls1[v] = 'nan'
      
  if len(nulls1) == 0: 
    nulls1 = t2[_list]
    for i in range(len(nulls1)):
      if nulls1[i] == 'nan':
        nulls1[i] = 0
  
  for i in range(len(nulls1)):
    if nulls1[i] == 'nan':
       nulls1[i] = None

  df.insert(19, down[_list], nulls1)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3249: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
df.insert(0, 'gdp', data['gdp'])
df.insert(0, 'population', data['population'])
df.insert(0, 'year', data['year'])

In [ ]:
df.isnull().sum()

year                                            0
population                                      0
gdp                                             3
greenhouse_gas_emissions                        0
electricity_demand                              0
                                               ..
other_renewables_elec_per_capita_exc_biofuel    0
other_renewables_energy_per_capita              0
other_renewables_share_elec                     0
other_renewables_share_elec_exc_biofuel         0
other_renewables_share_energy                   0
Length: 125, dtype: int64

In [ ]:
#gdp nulls using increase formula
y = 2018 - 1900
f = 1.814065e+13
s = 6.139981e+11

g = []
aagr = ((f/s)**(1/y)-1)*100
p = aagr/100
for i in range(3):
  x_0 = (f/(1-p))
  f = x_0 
  g.append(x_0)

gdp = data['gdp'].values.tolist()


for i in range(3):
  gdp.pop(-1)
gdp = gdp + g

df['gdp'] = gdp

In [ ]:
#energypergdp nulls using decrease formula
y = 2018 - 1900
f = 1.475000
s = 7.665222

g = []
aagr = ((f/s)**(1/y)-1)*100
p = aagr/100
for i in range(3):
  x_0 = (f/(1-p))
  f = x_0 
  g.append(x_0)

egdp = df['energy_per_gdp'].values.tolist()


for i in range(3):
  egdp.pop(-1)
egdp = egdp + g

df['energy_per_gdp'] = egdp

In [ ]:
print(df.isnull().sum().tolist(), end='')

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
df

,year,population,gdp,greenhouse_gas_emissions,electricity_demand,electricity_generation,per_capita_electricity,net_elec_imports,net_elec_imports_share_demand,energy_cons_change_pct,energy_cons_change_twh,energy_per_capita,energy_per_gdp,primary_energy_consumption,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,biofuel_electricity,biofuel_share_elec,biofuel_share_energy,coal_share_energy,coal_share_elec,coal_production,coal_prod_per_capita,coal_prod_change_twh,coal_prod_change_pct,coal_electricity,coal_elec_per_capita,coal_consumption,coal_cons_per_capita,coal_cons_change_twh,coal_cons_change_pct,carbon_intensity_elec,nuclear_cons_change_pct,nuclear_cons_change_twh,nuclear_consumption,nuclear_elec_per_capita,nuclear_electricity,nuclear_energy_per_capita,nuclear_share_elec,nuclear_share_energy,oil_cons_change_pct,oil_cons_change_twh,oil_consumption,oil_elec_per_capita,oil_electricity,oil_energy_per_capita,oil_prod_change_pct,oil_prod_change_twh,oil_prod_per_capita,oil_production,oil_share_elec,oil_share_energy,fossil_cons_change_pct,fossil_cons_change_twh,fossil_elec_per_capita,fossil_electricity,fossil_energy_per_capita,fossil_fuel_consumption,fossil_share_elec,fossil_share_energy,gas_cons_change_pct,gas_cons_change_twh,gas_consumption,gas_elec_per_capita,gas_electricity,gas_energy_per_capita,gas_prod_change_pct,gas_prod_change_twh,gas_prod_per_capita,gas_production,gas_share_elec,gas_share_energy,solar_cons_change_pct,solar_cons_change_twh,solar_consumption,solar_elec_per_capita,solar_electricity,solar_energy_per_capita,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,hydro_cons_change_pct,hydro_cons_change_twh,hydro_consumption,hydro_elec_per_capita,hydro_electricity,hydro_energy_per_capita,hydro_share_elec,hydro_share_energy,low_carbon_cons_change_pct,low_carbon_cons_change_twh,low_carbon_consumption,low_carbon_elec_per_capita,low_carbon_electricity,low_carbon_energy_per_capita,low_carbon_share_elec,low_carbon_share_energy,renewables_cons_change_pct,renewables_cons_change_twh,renewables_consumption,renewables_elec_per_capita,renewables_electricity,renewables_energy_per_capita,renewables_share_elec,renewables_share_energy,other_renewable_consumption,other_renewable_electricity,other_renewable_exc_biofuel_electricity,other_renewables_cons_change_pct,other_renewables_cons_change_twh,other_renewables_elec_per_capita,other_renewables_elec_per_capita_exc_biofuel,other_renewables_energy_per_capita,other_renewables_share_elec,other_renewables_share_elec_exc_biofuel,other_renewables_share_energy
0,1900,78763706.0,6.139981e+11,41.070188,445.780144,406.532120,3103.067242,6.312074e+07,1.422766e+07,-487.966805,-96754.466905,45700.771511,7.665222,5199.655902,-83.375212,-198.974926,0.033219,0.004515,143.614382,21.721467,5.478047,0.000401,19.534233,0.416376,1829.395000,23226.369000,297.871294+0.000000j,17.710173+0.000000j,1.691537,12.911644,1015.802276,8928.097007,-8730.816416,-21.538698,100.602300,-364546.233913,-95.687915,0.068584,0.181881,0.020693,0.602803,102.012763,0.001331,-121.709471,-23116.503477,1988.446789,87.851025,11.515734,17476.841193,2.951862+0.000000j,2.6695080+0.0000000j,1252.425000,98.646000,2.875288,38.236443,-1616.154143,-450896.714573,2272.328920,297.691878,48875.275269,5560.842094,73.240641,106.951058,-1.081792e+08,-2.856365e+09,2213.611861,1988.505445,260.507482,19455.861033,11.987244+0.000000j,7.9880000+0.0000000j,937.493000,73.840000,64.141983,42.563107,0.688509,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.103192e+06,0.000,0.000,0.000,0.000,0.000,0.000,-5.038843e-13,-6.374333e-11,1224.441952,3116.071976,354.531818,10761.932437,7495.063490,23.563999,1.086937e-15,0.000140,67.825608,195.581493,22.252361,596.130986,24.353508,1.304403,9.016709e-08,0.003888,83.841546,260.894541,29.683608,736.897809,1.519021,1.612203,6.889116,2.042509,2

In [ ]:
imaginary = ['coal_prod_change_twh', 'coal_prod_change_pct', 'oil_prod_change_pct', 'oil_prod_change_twh', 'gas_prod_change_pct', 'gas_prod_change_twh']

In [ ]:
#Remove the imaginary portion of the number and replace with the real portion
for i in range(len(imaginary)):
  arr1 =[]
  arr = np.array(data[imaginary[i]].values)
  for j in range(len(arr)): 
    r = (complex(arr[j]))
    arr1.append(r.real)
  df[imaginary[i]] = arr1

In [ ]:
df

,year,population,gdp,greenhouse_gas_emissions,electricity_demand,electricity_generation,per_capita_electricity,net_elec_imports,net_elec_imports_share_demand,energy_cons_change_pct,energy_cons_change_twh,energy_per_capita,energy_per_gdp,primary_energy_consumption,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,biofuel_electricity,biofuel_share_elec,biofuel_share_energy,coal_share_energy,coal_share_elec,coal_production,coal_prod_per_capita,coal_prod_change_twh,coal_prod_change_pct,coal_electricity,coal_elec_per_capita,coal_consumption,coal_cons_per_capita,coal_cons_change_twh,coal_cons_change_pct,carbon_intensity_elec,nuclear_cons_change_pct,nuclear_cons_change_twh,nuclear_consumption,nuclear_elec_per_capita,nuclear_electricity,nuclear_energy_per_capita,nuclear_share_elec,nuclear_share_energy,oil_cons_change_pct,oil_cons_change_twh,oil_consumption,oil_elec_per_capita,oil_electricity,oil_energy_per_capita,oil_prod_change_pct,oil_prod_change_twh,oil_prod_per_capita,oil_production,oil_share_elec,oil_share_energy,fossil_cons_change_pct,fossil_cons_change_twh,fossil_elec_per_capita,fossil_electricity,fossil_energy_per_capita,fossil_fuel_consumption,fossil_share_elec,fossil_share_energy,gas_cons_change_pct,gas_cons_change_twh,gas_consumption,gas_elec_per_capita,gas_electricity,gas_energy_per_capita,gas_prod_change_pct,gas_prod_change_twh,gas_prod_per_capita,gas_production,gas_share_elec,gas_share_energy,solar_cons_change_pct,solar_cons_change_twh,solar_consumption,solar_elec_per_capita,solar_electricity,solar_energy_per_capita,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,hydro_cons_change_pct,hydro_cons_change_twh,hydro_consumption,hydro_elec_per_capita,hydro_electricity,hydro_energy_per_capita,hydro_share_elec,hydro_share_energy,low_carbon_cons_change_pct,low_carbon_cons_change_twh,low_carbon_consumption,low_carbon_elec_per_capita,low_carbon_electricity,low_carbon_energy_per_capita,low_carbon_share_elec,low_carbon_share_energy,renewables_cons_change_pct,renewables_cons_change_twh,renewables_consumption,renewables_elec_per_capita,renewables_electricity,renewables_energy_per_capita,renewables_share_elec,renewables_share_energy,other_renewable_consumption,other_renewable_electricity,other_renewable_exc_biofuel_electricity,other_renewables_cons_change_pct,other_renewables_cons_change_twh,other_renewables_elec_per_capita,other_renewables_elec_per_capita_exc_biofuel,other_renewables_energy_per_capita,other_renewables_share_elec,other_renewables_share_elec_exc_biofuel,other_renewables_share_energy
0,1900,78763706.0,6.139981e+11,41.070188,445.780144,406.532120,3103.067242,6.312074e+07,1.422766e+07,-487.966805,-96754.466905,45700.771511,7.665222,5199.655902,-83.375212,-198.974926,0.033219,0.004515,143.614382,21.721467,5.478047,0.000401,19.534233,0.416376,1829.395000,23226.369000,NaN,NaN,1.691537,12.911644,1015.802276,8928.097007,-8730.816416,-21.538698,100.602300,-364546.233913,-95.687915,0.068584,0.181881,0.020693,0.602803,102.012763,0.001331,-121.709471,-23116.503477,1988.446789,87.851025,11.515734,17476.841193,NaN,NaN,1252.425000,98.646000,2.875288,38.236443,-1616.154143,-450896.714573,2272.328920,297.691878,48875.275269,5560.842094,73.240641,106.951058,-1.081792e+08,-2.856365e+09,2213.611861,1988.505445,260.507482,19455.861033,NaN,NaN,937.493000,73.840000,64.141983,42.563107,0.688509,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.103192e+06,0.000,0.000,0.000,0.000,0.000,0.000,-5.038843e-13,-6.374333e-11,1224.441952,3116.071976,354.531818,10761.932437,7495.063490,23.563999,1.086937e-15,0.000140,67.825608,195.581493,22.252361,596.130986,24.353508,1.304403,9.016709e-08,0.003888,83.841546,260.894541,29.683608,736.897809,1.519021,1.612203,6.889116,2.042509,2.186391,-52028.232260,-40384.481579,17.953191,14.471320,60.548605,1.990575,0.503023,0.132558
1,190

In [ ]:
df['coal_prod_change_twh'] = df['coal_prod_change_twh'].fillna(297.871294)
df['coal_prod_change_pct'] = df['coal_prod_change_pct'].fillna(17.710173)
df['oil_prod_change_pct'] = df['oil_prod_change_pct'].fillna(2.951862)
df['oil_prod_change_twh'] = df['oil_prod_change_twh'].fillna(2.6695080)
df['gas_prod_change_pct'] = df['gas_prod_change_pct'].fillna(11.987244)
df['gas_prod_change_twh'] = df['gas_prod_change_twh'].fillna(7.9880000)

In [ ]:
print(df.isnull().sum().tolist())

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
df.to_csv("/content/drive/MyDrive/Artificial Intelligence Thinking/Projects/US_final.csv")